In [ ]:
# LWVB: Larry Williams Volatility Breakout

import time
import datetime
import pandas as pd
import numpy as np
import os
import shutil
import plotly.express as px
import pprint
import glob

import itertools
import tqdm

In [ ]:
# variable parameters
rate_fee = 0.003
rate_slippage = 0.000
rate_cost = rate_fee + rate_slippage

param_open_time_hour = 00
param_open_time_min = 00
param_count_invest_crypto = 3
param_target_volatility = 0.05

In [ ]:
targetDataFiles = []

# specifying the path to csv files
path = "data/"

# csv files in the path
files = glob.glob(path + "/Upbit_*_min_*.csv")

# checking all the csv files in the 
# specified path
for filename in files:
    targetDataFiles += [filename]

print(targetDataFiles)

In [ ]:
targetDataFiles = targetDataFiles[:1]
targetDataFiles

In [ ]:
import ccqt

class BtEngine(ccqt.backtest.backtest):
    pass

be = BtEngine('backtest_LWVB2')

be.load_config('config.ini')
# be.prepare_slack('#lwvb-vn')
# be.check_slack()

In [ ]:


be.set_timeframe('minute')
for filename in targetDataFiles:
    ticker = filename.split('_')[1]
    be.load_ohlcv_min(ticker, filename)

be.infer_starttime_from_ohlcv()
be.infer_lasttime_from_ohlcv()
be.prepare_progress_bar()


In [ ]:
# 처음 틀었을때: 다음 첫 개장할때까지 기다리기
# 오늘 개장시각 기준으로 다음첫개장시각 확인
now = be.get_current_time()
first_open_time = datetime.datetime(
    now.year, now.month, now.day, 
    param_open_time_hour, 
    param_open_time_min, 
    0, 
    tzinfo=datetime.timezone.utc)
if now < first_open_time:
    be.loginfo("Backtest started before lwvb first open time {}".format(first_open_time))
else:
    be.loginfo("Backtest started after {}".format(first_open_time))
    first_open_time += datetime.timedelta(days=1)
    be.loginfo("first open time would be {}".format(first_open_time))


# 다음첫개장할때까지 대기
be.loginfo("Before first trade, now, wait until {}".format(first_open_time))
be.standby_until(first_open_time)

be.loginfo("now going to while loop...")


In [ ]:
print(be.m_curtime)
print(be.m_ticker2ohlcv_min.keys())
print(be.m_ticker2ohlcv_hour.keys())
print(be.m_timeframe)
print(be.m_balance)

In [ ]:
next_open_time = first_open_time 
# 세팅모드 먼저 시작하도록 한다!

tqdm.tqdm
ticker = 'TEST'

while be.check_loop():
    # loop invariant:
    # 1. next_open_time = cur_open_time + 1day
    # (not now) 2. cur_open_time <= now < next_open_time 
    # 3. cur_open_time < clear_time < next_open_time
    now = be.get_current_time()

    # 하루의 끝이자 시작. 세팅 모드
    if next_open_time <= now:
        be.loginfo(f'투자일 시작/종료 시각 도달, LWVB 초기 세팅 시작. {now}')
        cur_open_time = next_open_time
        next_open_time = next_open_time + datetime.timedelta(days=1)
        clear_time = next_open_time - datetime.timedelta(minutes=10)
        is_buy_completed = False
        is_sold_completed = False

        # MT여부 확인
        df = be.get_current_ohlcv_hour(ticker)
        hist_day = df.resample('1D', offset=datetime.timedelta(
            hours=param_open_time_hour,
            minutes=param_open_time_min)).agg(
        {
            'open': 'first',
            'close': 'last',
            'high': 'max',
            'low': 'min',
            'volume': 'sum',
            'value': 'sum',
        })
        if cur_open_time not in hist_day.index:
            be.logerror("cur_open_time not in hist_day.index")
            be.logerror(f'{cur_open_time=}')
            be.logerror(hist_day.index)

        hist_day['is_bull'] = True
        for open_dcount in [3, 5, 10]:
            colStr = f'maopen_{open_dcount}'
            hist_day[colStr] = hist_day['open'].shift(1).rolling(open_dcount).mean()
            hist_day['is_bull'] = hist_day['is_bull'] & (hist_day['open'].shift(1) > hist_day[colStr])

        be.loginfo(str(hist_day[['open', 'high', 'low', 'close', 'is_bull']]))
        if not hist_day['is_bull'][-1]: 
            be.loginfo("MT 조건 만족 안함! 오늘 하루 쉼")
            be.standby_until(next_open_time)
        
        # 기억하자:
          # hist_day 에서 마지막행은 오늘 데이터(시가만 유효)이고
          # 마지막 전 행이 어제 데이터다

        # 아래 코드에서 prev: 어제를 의미
        range_prev = hist_day['high'][-2] - hist_day['low'][-2]
        noise_prev = 1 - abs(hist_day['close'][-2] - hist_day['open'][-2]) / range_prev
        breakout_price = hist_day['open'][-1] + range_prev * noise_prev
        volatility = range_prev / hist_day['open'][-2]
        if volatility <= param_target_volatility:
            invest_ratio = 1
        else:
            invest_ratio = param_target_volatility / volatility

        today_open = hist_day['open'][-1]
        be.loginfo(f'{range_prev=}, {noise_prev=}, {breakout_price=}, {volatility=}, {invest_ratio=}, {today_open=}')



    # loop invariant:
    # 1. next_open_time = cur_open_time + 1day
    # 2. cur_open_time <= now < next_open_time
    assert(cur_open_time <= now < next_open_time)
    # 3. cur_open_time < clear_time < next_open_time
    assert(cur_open_time < clear_time < next_open_time)
    
    # now loop invariant 2. holds
    
    if clear_time <= now :
        # 하루의 종료에 다다름 정리 모드
        # 모두 팔고, 수익실현, 수익률계산하고, 현재 관리예금 업데이트
        if is_sold_completed:
            be.logerror('매도 이미 완료, 다음 개장 시간 까지 대기 ...')
        else:
            be.loginfo("Starting Selling...")
            btc_amount = be.get_current_balance(ticker)
            be.order_sell_market(ticker, btc_amount)
            is_sold_completed = True

        be.report_end_of_day()
        be.standby_until(next_open_time)

    # 거래 모드
    if now < clear_time:
        if is_buy_completed:
            be.loginfo('매수 이미 완료, 정리 시간 까지 대기 ...')
            be.standby_until(clear_time)
        else:
            curPrice = be.get_current_price(ticker)
            if breakout_price <= curPrice:
                be.loginfo("Starting Buying...")
                be.order_buy_market_including_fee(ticker, be.get_current_balance('KRW') * invest_ratio)
                is_buy_completed = True
                be.standby_until(clear_time)
    be.standby_short()

print("Backtest done!")


아래는 예시 코드

In [ ]:
be.m_result_df

In [ ]:
a = datetime.datetime(2022, 2, 3, tzinfo=datetime.timezone.utc)
a.date()

df_columns = [
    'is_bull',
    'cagr(%)_all',
    'cagr(%)_3month',
    'cagr(%)_month',
    'cagr(%)_week(7d)',
    'final_asset_in_krw',
    'stdev(%)_all',
    'mdd(%)',
    'recovery_days',
    'ration_win(%)',
    'mean_profit(%)',
    'mean_loss(%)',
    'max_profit(%)',
    'max_loss(%)',
    'count_trade',
    'count_all',
]
df = pd.DataFrame(
    columns=df_columns,
    index=pd.to_datetime([], utc=True))

df.loc[a] = 3
df